<a href="https://colab.research.google.com/github/ailton-felix/Combinacoes/blob/master/nmt/nmt_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Projeto da Máquina de Tradução NMT (Neural Machine Translation) **2.0**. Utilizamos o modelo word2word que é estruturado para traduzir palavra por palavra onde a predição do próximo depende da obtenção da palavra anterior.* 
<div align="center">
 <img src="https://machinelearningexps.appspot.com/static/images/hugo_nmt.png">
</div>

## Definições
Contém definições sucintas sobre Máquina de Tradução por Rede Neural.

### Bleu
Bleu é um número utilizado para definir a qualidade da tradução

Pontuação BLEU | Interpretação
-- | --
< 10 | Praticamente inútil
10 - 19 | Difícil de compreender o sentido
20 - 29 | O sentido está claro, mas há erros gramaticais graves
30 - 40 | Pode ser entendido como boas traduções
40 - 50 | Traduções de alta qualidade
50 - 60 | Traduções de qualidade muito alta, adequadas e fluentes
60 > | Em geral, qualidade superior à humana
> [Fonte: Google](https://cloud.google.com/translate/automl/docs/evaluate)

# Instalar dependências

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import auth, output
import os
import sys

In [4]:
print("GCLOUD AUTH")
auth.authenticate_user()

GCLOUD AUTH


instalar NMT

In [7]:
os.environ['GOOGLE_CLOUD_PROJECT'] = "machinelearningexps"
!gcloud source repos clone github_hand-talk_nmt --project=machinelearningexps
!mv github_hand-talk_nmt nmt
!pip install tensorflow==2.2.0
output.clear()

instalar pytools

In [12]:
!gcloud source repos clone github_hand-talk_pytools --project='ht-community' && pip install /content/github_hand-talk_pytools
output.clear()

configurar versão da nmt

In [8]:
assert os.path.exists('./nmt'), 'Erro! Você precisar clonar o repositório da NMT aqui!'

!cd ./nmt/ && echo branch atual: && git rev-parse --abbrev-ref HEAD
version = input('Versão/branch da nmt (ENTER para a atual):')
if version:
    !cd ./nmt/ && git checkout {version} && git pull
    print(f'versão da nmt atualizada para: {version}')
    
# adicionando nmt    
if './nmt' not in sys.path:
    sys.path.insert(0, './nmt')
    
# adicionando pytools
# if './pytools' not in sys.path:
#     sys.path.insert(0, './pytools')
print('Path do sistema:')
sys.path[:2]

branch atual:
master
Versão/branch da nmt (ENTER para a atual):
Path do sistema:


['./nmt', '']

In [13]:
import json
import sys
import tensorflow as tf
import numpy as np
import pandas as pd
import cereja as cj
import random
from datetime import datetime
from matplotlib import pyplot
from copy import copy

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# # ------- CUIDADO ISSO QUEBRA A VM -------:
# !pip install tornado==4.5.3 #-> resolve problema do asyncio no jupyter
# #se for usar, execute o comando abaixo para restaurar a versão do tornado:
# # !pip install tornado -U
# # ------- CUIDADO ISSO QUEBRA A VM -------:

🍒 Using Cereja v.1.4.8


# Hiperparâmetros da NMT

In [14]:
translation_languages_c = ["en-asl", "pt-bzs"]
preditor_c = ['BaselineNgramPredict', "SequentialPredict"]
model_architecture_c = ["HtNmt", "HTModel", "AttentionModel"]
optimizer_c = ["adam", "SGD", "RMSprop", "Adadelta", "Adagrad", "Adamax", "Nadam", "Ftrl"] 
learning_rate_c = ["0.003", "0.00001", "0.0001", "0.001", "0.01", "0.03", "0.1", "0.3", "1", "3", "10"]
decay_c = ["0.0002", "0", "0.003", "0.01", "0.03", "0.1", "1", "3", "10"]
test_from_c = ['legacy', 'new data', 'mix', 'input']

    
@interact
def comment_input(Comment='Treinamento da NMT ...'):
    global comment_of_train
    comment_of_train=Comment
    
    
@interact
def hyperparameters_list(Languages=translation_languages_c, Predictor=preditor_c, Model=model_architecture_c, 
                         Optimizer=optimizer_c, Learning=learning_rate_c, Decay=decay_c,
                         Test_from=test_from_c):
    global translation_languages, preditor, model_architecture
    global optimizer, learning_rate, decay, use_test_from
    translation_languages=Languages
    preditor=Predictor
    model_architecture=Model
    optimizer=Optimizer
    learning_rate=Learning
    decay=Decay
    use_test_from=Test_from
    
    
@interact
def hyperparameters_input(Epochs='30', Batch_size='32', Dimensions='100',
                          Units='128', NTR='0.75', Sentence_len='10'):
    global n_epochs, batch_size, n_dimensions, n_units, ntr, max_length_sentence
    n_epochs=int(Epochs) if Epochs else 30
    batch_size=int(Batch_size) if Batch_size else 32
    n_dimensions=int(Dimensions) if Dimensions else 10
    n_units=int(Units) if Units else 128
    ntr=float(NTR) if NTR else 0.75
    max_length_sentence=int(Sentence_len) if Sentence_len else 10
    
    
@interact
def hyperparameters_bool(use_dictionary=True, run_quality_test=False, run_performance_test=False, 
                         run_save_model=True, use_extra_vocab=True):  
    global use_dict, extra_vocab, quality_test, performance_test, save_model
    use_dict=use_dictionary
    quality_test=run_quality_test
    performance_test=run_performance_test
    save_model=run_save_model
    extra_vocab=use_extra_vocab

interactive(children=(Text(value='Treinamento da NMT ...', description='Comment'), Output()), _dom_classes=('w…

interactive(children=(Dropdown(description='Languages', options=('en-asl', 'pt-bzs'), value='en-asl'), Dropdow…

interactive(children=(Text(value='30', description='Epochs'), Text(value='32', description='Batch_size'), Text…

interactive(children=(Checkbox(value=True, description='use_dictionary'), Checkbox(value=False, description='r…

In [16]:
print(f'Treinamento da NMT dados convertidos mais dados community mais Palavras Relacionadas + teste {use_test_from} + {n_epochs} epocas + {max_length_sentence} input + {n_units} neurônios + {learning_rate} LR + {ntr} ntr')
# Treinamento da NMT somente com dados convertidos + teste input + 30 epocas + 10 input + 128 neurônios + 0.003 LR + 0.75 ntr

Treinamento da NMT dados convertidos mais dados community mais Palavras Relacionadas + teste legacy + 30 epocas + 10 input + 128 neurônios + 0.003 LR + 0.75 ntr


In [17]:
print(f'Treinamento para tradução: {translation_languages}')
year, month, day, hour_min = datetime.today().strftime('%Y-%m-%d-%H_%M').split('-')
source_language, target_language = translation_languages.split('-')

job_dir = f"/content/nmt/JOB_DIR/{source_language}_{target_language}"
job_dir_exists = os.path.exists(job_dir)

if not job_dir_exists:
    os.makedirs(job_dir)
else:
    assert(input(f'Todo conteúdo de {job_dir} será apagado. Digite "sim" para prosseguir').lower() == 'sim')
    !rm -r '{job_dir}'/*

NMT_CONFIG = {
  "_comment": "Configuração geral do sistema - NEW",
  "SOURCE_LANGUAGE": source_language,
  "TARGET_LANGUAGE": target_language,
  "ENVIRONMENT": "development",
  "CHOICE_PREDICTS": {
    "Sequential": f"nmt.core.predict.predictors.{preditor}"
  },
  "MODEL_ARCHITECTURE": f"nmt.core.models.base.{model_architecture}",
  "EPOCHS": n_epochs,
  "JOB_DIR": job_dir,
  "CORPUS_DIR": "/content/CORPUS_DIR",
  "NORMAL_TRAIN": True,
  "PERFORMANCE_TEST": True,
  "ATTENTION": "ht_attention",
  "USE_DICT": use_dict,
  "OPTIMIZER": optimizer,
  "LEARNING_RATE": float(learning_rate),
  "DECAY": float(decay),
  "N_DIM": n_dimensions,
  "N_UNITS": n_units,
  "BATCH_SIZE": batch_size,
  "EXTRA_VOCAB": extra_vocab,
  "NTR": ntr,
  "LOG_ARGMAX_LENGTH": 3,
  "MAX_SOURCE_LENGTH_SENTENCE": max_length_sentence,
  "MAX_TARGET_LENGTH_SENTENCE": max_length_sentence,
  "PARSER": max_length_sentence,
}

with open('./nmt/nmt/config/default_config.json', 'w', encoding='utf-8') as f:
    json.dump(NMT_CONFIG, f, indent=True)

import nmt
nmt.config.settings.update_settings(NMT_CONFIG)
cj.FileIO.create(os.path.join(nmt.config.settings.JOB_DIR, 'nmt_config.json'), NMT_CONFIG).save()

Treinamento para tradução: en-asl


In [18]:
from nmt.core.pln.corpus_toolkit import Corpus, restrict_sentences
from nmt.config import settings
from nmt.core.predict import BaselineNgramPredict
from nmt.core.tokenizer.tokenizer import Vectorizer
from nmt.core.predict.controller import Translator
from nmt.core.pln import PreProcessing, PostProcessing
from nmt.core.pln import operations
from nmt.core.models.base import HtNmt

seed = nmt.config.settings.NMT_TRAIN_CONFIG['random_seed']
random.seed(seed)
np.random.seed(seed)
tf.compat.v1.set_random_seed(seed)
ON_ASL = True if settings.TARGET_LANGUAGE == 'asl' else False

# Preprocessamento
Agora vamos realizar o preprocessamento dos dados.

1 - Você pode simplesmente utilizar a configuração predefinida.

2 - Ou construir seu preprocessamento
Para construir seu preprocessamento utilizando as operações você vai precisar entender um pouco a nossa estrutura.

In [19]:
preprocess = PreProcessing.build(preprocessing_config=settings.PLN_CONFIG)
# para visualizar as operações ativas

#### HABILITAR SE QUISER STEMMAR
# preprocess.add_operation('stemming', language = 'en')
######
preprocess.operations

[LowerCaseSelect,
 FilterSignals,
 RemoveStopWords,
 RemoveItems,
 JoinAlphaNumeric,
 ReplaceEnglishContractions,
 RemoveAccents]

## Adicionando operações ao preprocessamento (Opcional)
Para adicionar novas operações utilize o método `add_operation`

Por padrão já desenvolvemos algumas operações de preprocessamento, você pode visualiza-las utilizando o método `nmt.core.pln.operations.look_up`

Você pode facilmente criar a sua e acoplar a nmt para isso utilizando a classe `nmt.core.pln.operations.Operation`. É necessário criar o método abstrato `op`, onde ele recebe uma string (frase) e deve retornar uma string também


Exemplo:
```python
class LowerCase(operations.Operation):

    def op(self, sentence: str) -> str:
        # corpo da sua função para a manipulação da sentença recebida
        return sentence.lower()

preprocess.add_operation(LowerCase, is_only_target=True, exec_priority=0)
```


A classe `Operation` tem os seguintes atributos 

* `is_only_source: bool = False` como o seu nome sugere a operação só será executada na sentença fonte

* `is_only_target: bool = False` o mesmo acontece em relação as sentenças alvo. Caso ambas as flags forem `False` significa que é uma operação para ambas as línguas 

* `exec_priority: int = -1` as operações são executadas com uma determinada prioridade, isso é necessário em alguns casos, tenha isso em mente




In [32]:
def preprocessing_community_data(corpus: Corpus, preprocess=None):
    """
    Esta função resolve alguns problemas que foram identificados no community,
    ela é temporária até o problema ser resolvido na raiz
    """
    assert isinstance(corpus, Corpus), "Você precisa enviar uma instância de 'nmt.core.pln.corpus_toolkit.Corpus'"
    # Espaço para você criar suas operações (:
    if corpus.source_language == 'en' and corpus.target_language == 'asl':
        new_source = []
        new_target = []
        total_removed = 0
        total_fixed = 0
        for i, (x, y) in enumerate(corpus):
            prep_x = preprocess.clean(x).split()
            prep_y = preprocess.clean(y).split()
            error = list(filter(lambda x: '¶' not in x and x not in prep_x, prep_y))
            if error:
                total_removed += 1
                continue
            if prep_x.count('tom') > 1 or prep_y.count('tom') > 1:
                total_removed += 1
                continue
            if prep_x.count('tom') != prep_y.count('tom'):
                total_removed += 1
                continue
            if ('mary' in prep_x or 'marys' in prep_x) and 'mari' in prep_y:
                total_fixed += 1
                y = y.replace('mari', 'mary')
                prep_y = preprocess.clean(y).split()
            if prep_x.count('mary') > 1 or prep_x.count('marys') > 1 or prep_y.count('mary') > 1:
                total_removed += 1
                continue
            if prep_x.count('mary') != prep_y.count('mary'):
                total_removed += 1
                continue
            if prep_y.count('?'):
                y = y.replace('?', '')
                total_fixed += 1
            if prep_y.count(','):
                y = y.replace(',', '')
                total_fixed += 1
            prep_x = preprocess.clean(x).split()
            prep_y = preprocess.clean(y).split()
            error = list(filter(lambda x: '¶' not in x and x not in prep_x, prep_y))
            if error:
                total_removed += 1
                continue
            
            new_source.append(x)
            new_target.append(y)

        print(f'total_removed = {total_removed}')
        print(f'total_fixed = {total_fixed}')
        return Corpus(new_source, new_target, source_name='en', target_name='asl', preprocessor=preprocess)
    return corpus


def load_replace_dict(file_path):
    assert os.path.exists(file_path), f'"{file_path}" não encontrado'
    with open(file_path) as f:
        original_replaces = json.load(f)
    return {f'¶{sign_key}': f'¶{sign_value}' if not isinstance(sign_value, list) else [f'¶{sign}' for sign in sign_value]
            for sign_key, sign_value in original_replaces.items()}


def replace_signs(corpus: Corpus, workspace='ASL', preprocess=None):
    replace_signs_path = f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/NMT/SUBSTITUICAO_DE_SINAIS/REPLACE_SIGNS_{workspace.upper()}.json'
    sign_new_sign = load_replace_dict(replace_signs_path)

    assert isinstance(corpus, Corpus), "Você precisa enviar uma instância de 'nmt.core.pln.corpus_toolkit.Corpus'"
    
    if corpus.source_language == 'en' and corpus.target_language == 'asl':
        new_source = []
        new_target = []
        total_replaced = 0
        for x, y in corpus:
            result = [sign_new_sign.get(sign, sign) for sign in y.split(' ')]
            total_replaced += sum(map(lambda w: w[0] != w[1], zip(y.split(' '), result)))                              
            y = ' '.join(result)
            new_source.append(x)
            new_target.append(y)
        print('total_replaced =', total_replaced)
        return Corpus(new_source, new_target, source_name='en', target_name='asl', preprocessor=preprocess)
    return corpus


def generate_variations_sentences(index, new_sentences, signs_with_variations, signs_variations):
    if index < len(signs_with_variations):
        sign = signs_with_variations[index]
        vars = signs_variations[sign] 
        len_sentences = len(new_sentences)
        sentences_temp = list(new_sentences)
        for i_sent in range(len_sentences):
            sentence = sentences_temp[i_sent]
            for var in vars:
                if var == sign:
                    continue
                new_sentences.append(sentence.replace(sign, var))
        generate_variations_sentences(index+1, new_sentences, signs_with_variations, signs_variations)


def add_variations_sentences(corpus: Corpus, workspace='ASL', preprocess=None):
    signs_variations_path = f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/NMT/SUBSTITUICAO_DE_SINAIS/SIGN_VARIATIONS_{workspace.upper}.json'
    signs_variations = load_replace_dict(signs_variations_path)

    assert isinstance(corpus, Corpus), "Você precisa enviar uma instância de 'nmt.core.pln.corpus_toolkit.Corpus'"
    
    if corpus.source_language == 'en' and corpus.target_language == 'asl':
        new_source = []
        new_target = []
        total_variations = 0
        for x, y in corpus:
            new_sentences = [y]
            signs_with_variations = list(filter(lambda signs: signs in signs_variations, y.split(' ')))
            if signs_with_variations:
                generate_variations_sentences(0, new_sentences, signs_with_variations, signs_variations)
                total_variations += len(new_sentences) - 1 
            new_source.extend([x]*len(new_sentences))
            new_target.extend(new_sentences)
        print('total_variations =', total_variations)
        assert len(new_source) == len(new_source), f'Erro! source {len(new_source)} != target {len(new_source)}'
        return Corpus(new_source, new_target, source_name='en', target_name='asl', preprocessor=preprocess)
    return corpus

# Corpus
Primeiro passo é carregar e dividir o corpus em dados de treino e teste

### Divisão dos dados 
Você pode carregar os dados direto do storage, sempre será carregado o último corpus gerado pelo community.

In [29]:
from typing import Tuple
@interact
def test_size(Test_percent=widgets.IntSlider(min=1, max=50, step=1, value=10)):
    global test_percent
    test_percent=Test_percent
    
# Gera o corpus com vocabulário reduzido (ex: para vocabulário 100, o algoritmo vai gerar 
# o corpus baseado neste vocabulário, sempre priorizando da mais frequente para a menos frequente)
source_vocab_size = 0
target_vocab_size = 0

# Usada para separar treino e teste do corpus, podendo considerar o conjunto de validação legado
def split_corpus(corpus: Corpus, test_percent: int=10, validation: Corpus=None) -> Tuple[Corpus, Corpus]:
    # para remoção do conjunto de teste, sentenças de validação
    validation_verify = {preprocess.clean(i) for i in validation.source.data} if validation \
                        else None
    test_max_size = int((test_percent/100) * len(corpus))
    train, test = corpus.split_data(test_max_size=test_max_size, 
                                    source_vocab_size=source_vocab_size, 
                                    target_vocab_size=target_vocab_size, 
                                    take_corpus_instances=True, 
                                    source_cannot_go_test=validation_verify)
    # validação - nenhuma sentença de validação pode estar no teste
    if validation_verify:
        print('Checando conjunto de validade em teste')
        for x, _ in test:
            assert preprocess.clean(x) not in validation_verify, f'Erro! Existe sentença de validação no conjunto de teste'
        print('Tudo certo! Pode seguir :D')
    return train, test

interactive(children=(IntSlider(value=10, description='Test_percent', max=50, min=1), Output()), _dom_classes=…

### Corpus Legado

In [30]:
source_column_name, target_column_name = ('sentence', 'parsedTags') if ON_ASL else ('source_sentence', 'target_sentence')
source_column_name, target_column_name 

('sentence', 'parsedTags')

In [33]:
corpus = Corpus.load_from_storage(f"CORPUS_{settings.TARGET_LANGUAGE.upper()}.csv",
                                  src_col_name=source_column_name,
                                  trg_col_name=target_column_name,
                                  source_name=settings.SOURCE_LANGUAGE,
                                  target_name=settings.TARGET_LANGUAGE,
                                  corpus_version='legacy',
                                  join_id_glosa=False)   

if ON_ASL:
    corpus_legacy, validations_legacy = corpus[0], corpus[1]
    print("Corrigindo Validações")
    validations_legacy = preprocessing_community_data(validations_legacy, preprocess=preprocess)
    # validations_legacy = add_variations_sentences(validations_legacy, preprocess=preprocess)        
    validations_legacy = replace_signs(validations_legacy, workspace=target_language, preprocess=preprocess)
    print("Corrigindo Corpus")
    corpus_legacy = preprocessing_community_data(corpus_legacy, preprocess=preprocess)
    # corpus_legacy = add_variations_sentences(corpus_legacy, preprocess=preprocess)
    corpus_legacy = replace_signs(corpus_legacy, workspace=target_language, preprocess=preprocess)
else:
    corpus_legacy = corpus
    validations_legacy = None

print('Corpus legado -', corpus_legacy)
print('Validação legado -', validations_legacy or 'Vazio')

ID_GLOSA_ASL.json baixado em: "/content/nmt/JOB_DIR/en_asl/ID_GLOSA_ASL.json"
Total of empty parallel sentences ignored: 147
Corrigindo Validações
total_removed = 466
total_fixed = 133
total_replaced = 52205
Corrigindo Corpus
total_removed = 159
total_fixed = 0
total_replaced = 4459
Corpus legado - Corpus(examples: 20848 - source_vocab_size: 11184 - target_vocab_size:15822)
Validação legado - Corpus(examples: 51361 - source_vocab_size: 5847 - target_vocab_size:5642)


In [34]:
if use_test_from == 'legacy':
    print('Usando teste: ', use_test_from)
    train_legacy, test = split_corpus(corpus_legacy, test_percent, validations_legacy)
    print('Teste final -', test)
else:
    train_legacy = corpus_legacy
    
print('Treino legado -', train_legacy)
# Treino legado - Corpus(examples: 204812 - source_vocab_size: 41589 - target_vocab_size:18171)

Usando teste:  legacy
Checando conjunto de validade em teste
Tudo certo! Pode seguir :D
Teste final - Corpus(examples: 1085 - source_vocab_size: 1219 - target_vocab_size:1187)
Treino legado - Corpus(examples: 19763 - source_vocab_size: 11305 - target_vocab_size:15825)


### Corpus de Sinais de Palavras Relacionadas
**Apenas para BZS**

In [35]:
train_related_words = []
if target_language == 'bzs':
    df_sentences = pd.read_csv('/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/NMT/SENTENCAS_PALAVRAS_RELACIONADAS/related_words_sentences_bzs.csv')
    source_list, target_list = df_sentences['source_sentence'], df_sentences['target_sentence']
    train_related_words = Corpus(source_list, target_list,
                                 source_name=settings.SOURCE_LANGUAGE,
                                 target_name=settings.TARGET_LANGUAGE,
                                 preprocess=preprocess)
print('Treino palavras relacionadas - ', train_related_words)

Treino palavras relacionadas -  []


### Novo Corpus
Com os novos dados do Community

In [38]:
corpus_version = None #'2021-03-17'

In [39]:
source_column_name, target_column_name = 'source_sentence', 'target_sentence'
new_corpus = Corpus.load_from_storage(f"CORPUS_{settings.TARGET_LANGUAGE.upper()}.csv",
                                      src_col_name=source_column_name,
                                      trg_col_name=target_column_name,
                                      source_name=settings.SOURCE_LANGUAGE,
                                      target_name=settings.TARGET_LANGUAGE,
                                      corpus_version=corpus_version,
                                      join_id_glosa=True)   

if ON_ASL:
    print("Corrigindo Corpus")
    new_corpus = preprocessing_community_data(new_corpus, preprocess=preprocess)
    # new_corpus_clean = add_variations_sentences(new_corpus_clean, preprocess=preprocess)
    new_corpus = replace_signs(new_corpus, workspace=target_language, preprocess=preprocess)
print('Novo corpus -', new_corpus)
# ID_GLOSA_ASL.json baixado em: "./JOB_DIR/en_asl/ID_GLOSA_ASL.json"
# Total of empty parallel sentences ignored: 0
# Corrigindo Corpus
# total_removed = 110
# total_fixed = 0
# total_replaced = 3872
# Novo corpus - Corpus(examples: 8284 - source_vocab_size: 4515 - target_vocab_size:4240)

Juntando com ID_GLOSA já existente: "/content/nmt/JOB_DIR/en_asl/ID_GLOSA_ASL.json"...
ID_GLOSA_ASL.json baixado em: "/content/nmt/JOB_DIR/en_asl/ID_GLOSA_ASL.json"
Total of empty parallel sentences ignored: 0
Corrigindo Corpus
total_removed = 139
total_fixed = 0
total_replaced = 11268
Novo corpus - Corpus(examples: 10523 - source_vocab_size: 4651 - target_vocab_size:4493)


In [40]:
if use_test_from == 'new data':
    print('Usando teste: ', use_test_from)
    new_train, test = split_corpus(new_corpus, test_percent, validations_legacy)
    print('Teste final -', test)
else:
    new_train = new_corpus
    
print('Novo Treino -', new_train)

Novo Treino - Corpus(examples: 10523 - source_vocab_size: 4651 - target_vocab_size:4493)


### Merge Corpus

É hora de juntar dados do corpus legado + validação + novos dados do community!

In [41]:
corpus_tuple = [(x, y) for x, y in validations_legacy] if ON_ASL else []
corpus_tuple += [(x, y) for x, y in train_legacy]
corpus_tuple += [(x, y) for x, y in train_related_words]
corpus_tuple += [(x, y) for x, y in new_train]

corpus_tuple = list({(x, y) for x, y in corpus_tuple})

corpus_source = []
corpus_target = []
for x, y in corpus_tuple:
    corpus_source.append(x)
    corpus_target.append(y)
assert len(corpus_source) == len(corpus_target), 'Erro! Sentenças fonte e alvo diferentes: ' \
                                               f'{len(corpus_source)} != {len(corpus_target)})'

corpus = Corpus(corpus_source, corpus_target,
               source_name=settings.SOURCE_LANGUAGE,
               target_name=settings.TARGET_LANGUAGE,
               preprocess=preprocess)

if use_test_from == 'mix':
    print('usando teste: ', use_test_from)
    train, test = split_corpus(corpus, test_percent, validations_legacy)
    print('Teste final -', test)
elif use_test_from == 'input':
    print('usando teste: ', use_test_from)
    # Informe o path do teste aqui:
    x_test_path = '/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/NMT/CORPUS_LEGADO_BZS/converted_test_without_leak_pt.align'
    y_test_path = '/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/NMT/CORPUS_LEGADO_BZS/converted_test_without_leak_bzs.align'
    assert os.path.exists(x_test_path), f'"{x_test_path}" não encontrado!'
    assert os.path.exists(y_test_path), f'"{y_test_path}" não encontrado!'
    with open(x_test_path) as f:
        x_test = f.read().split('\n')
    with open(y_test_path) as f:
        y_test = f.read().split('\n')
    assert len(x_test) == len(y_test), f'Erro! Base de treino inconsistente {len(x_test)} != {len(y_test)}'
    test = Corpus(x_test, y_test,
                  source_name=settings.SOURCE_LANGUAGE,
                  target_name=settings.TARGET_LANGUAGE,
                  preprocess=preprocess)
    train = corpus
    print('Teste final -', test)
else:
    train = corpus
    
print('Treino Final -', train)
# usando teste:  input
# Teste final - Corpus(examples: 1870 - source_vocab_size: 2978 - target_vocab_size:1747)
# Treino Final - Corpus(examples: 200287 - source_vocab_size: 42407 - target_vocab_size:20191)

Treino Final - Corpus(examples: 58610 - source_vocab_size: 13268 - target_vocab_size:17943)


In [42]:
# validação - verifica se existe sentença de treino, no conjunto de teste (leak)
count = 0
train_set = {preprocess.clean(x) for x,_ in train}
for src, trg in test:
    if preprocess.clean(src) in train_set:
        count+=1
print(f'Foram encontradas {count} sentenças de treino no teste\nNão se aperrei,', 
      'os leak serão removidos mais pra frente ;D') if count else \
print('Tudo certo! Pode seguir :D')
# Foram encontradas 136 sentenças de treino no teste
# Não se aperrei, os leak serão removidos mais pra frente ;D

Foram encontradas 702 sentenças de treino no teste
Não se aperrei, os leak serão removidos mais pra frente ;D


In [43]:
train.source.save_freq(settings.JOB_DIR, 'train_source_freq')
train.target.save_freq(settings.JOB_DIR, 'train_target_freq')
test.source.save_freq(settings.JOB_DIR, 'test_source_freq')
test.target.save_freq(settings.JOB_DIR, 'test_target_freq')

train.save(settings.JOB_DIR,take_split=False, shuffle=False, prefix='train')
test.save(settings.JOB_DIR, take_split=False, shuffle=False, prefix='test')

id_glosa_path = os.path.join(settings.JOB_DIR, f'ID_GLOSA_{settings.TARGET_LANGUAGE.upper()}.json')
with open(id_glosa_path, encoding='utf-8') as fp:
    settings.update_settings({f"ID_GLOSA_{settings.TARGET_LANGUAGE.upper()}":json.load(fp)})

print('Usando teste: ', use_test_from)
print('Treino -', train)
print('Teste -', test)

Usando teste:  legacy
Treino - Corpus(examples: 58610 - source_vocab_size: 13268 - target_vocab_size:17943)
Teste - Corpus(examples: 1085 - source_vocab_size: 1219 - target_vocab_size:1187)


## Executando o preprocessamento nos dados
Estamos quase chegando no treinamento, já temos um *Corpus*, um *Preprocessador*



```
Corpus + Preprocessador = Dados lindos
```

Vamos definir o número máximo de ***palavras desconhecidas*** em uma sentença, isso mesmo o famoso `UNK`.

Não podemos definir um número maior que o número máximo de tokens que nossa rede suporta, então vamos utilizar essa infomação que já temos na configuração da nmt.

Simplesmente pegamos o mínimo entre o tamanho máximo que a rede suporta entre a sentença fonte e a sentença alvo

Depois de definir o número de unk's nós precisamos passar esse valor como parâmetro para a função do ***preprocessador*** `clean_data_set` e como o nome sugere, é responsável por executar as ***operações*** comuns e as específicas em todos nossos dados

In [44]:
# Aqui está nosso número de palavras desconhecidas.
n_unks = min(settings.MODEL_STRUCTURE["max_source_length_sentence"],
             settings.MODEL_STRUCTURE["max_target_length_sentence"])

# Muito bom! Chegou a hora de colocar o preprocessador para funcionar.
cleaned_train = preprocess.clean_data_set(train, n_unks=n_unks, fill_unk=True, verbose=True)
cleaned_test = preprocess.clean_data_set(test, n_unks=n_unks, fill_unk=False, verbose=True)

# Remove LEAK
print("Verificando se contém LEAK depois do preprocessamento")
x = {src:index for index, (src, trg)  in enumerate(cleaned_train)}
y = {src:index for index, (src, trg)  in enumerate(cleaned_test)}
index_leaks = [x[i] for i in set(y).intersection(set(x))]
if index_leaks:
    print(f"Encontrou {len(index_leaks)} LEAKS.")
    print(f"Tamanho do treino antes da remoção: {len(cleaned_train)}")
    cleaned_train = np.array([train_line for i, train_line in enumerate(cleaned_train) if i not in index_leaks])
    print(f"Tamanho do treino depois da remoção: {len(cleaned_train)}")
else:
    print('Nenhum LEAK foi encontrado...oia que bom :D')

# Restringe as sentenças fonte e alvo do conjunto de dados.
cleaned_train = restrict_sentences(parallel_data=cleaned_train,
                                   source_seq_length=settings.NMT_TRAIN_CONFIG['model_structure'][
                                       'max_source_length_sentence'],
                                   target_seq_length=settings.NMT_TRAIN_CONFIG['model_structure'][
                                       'max_target_length_sentence'])

# BZS
# 1867/18707747Verificando se contém LEAK depois do preprocessamento
# Encontrou 155 LEAKS.
# Tamanho do treino antes da remoção: 197757
# Tamanho do treino depois da remoção: 197602

1084/1085Verificando se contém LEAK depois do preprocessamento
Encontrou 541 LEAKS.
Tamanho do treino antes da remoção: 58620
Tamanho do treino depois da remoção: 58079


# Tokenizador - transformando frases em uma sequência de números

A partir de agora entramos no mundo dos números!

Nós iremos tokenizar nossos dados, onde palavras tornam-se números.

Para isso vamos utilizar a classe `Vectorizer` e vamos passar os dados preprocessados, junto com o número de ***unk's***

**Importante!!!**
O tokenizer é a chave para codificação e decodificação, então você vai querer guardá-lo.

Calma, você vai ver que nós facilitamos tudo isso para você.

In [45]:
# Antes de iniciar com o Vectorizer, vamos concatenar para facilitar a busca
# Outra coisa muito importante!!! Nunca manipule string com numpy, você vai perder informação.
# Se for utilizar pelas facilidades e costume, utilize apenas como leitura

# Para acessar os dados da sentença fonte use `data[:,0]` e `data[:,1]` para acessar os dados da sentença alvo.
data = np.concatenate([cleaned_train, cleaned_test])

# Construção do path do arquivo onde o tokenizer (.txt) será salvo.
# Nós salvamos tudo que é gerado no diretório JOB_DIR/<lingua_fonte>_<lingua_alvo>/ é regra.
# Inclusive já criamos ele pra você sem você nem perceber rsrs.
source_tokenizer_path = os.path.join(settings.JOB_DIR, f'{settings.SOURCE_LANGUAGE}.tk')
target_tokenizer_path = os.path.join(settings.JOB_DIR, f'{settings.TARGET_LANGUAGE}.tk')

# Aqui você cria os tokenizers passando o path de onde será salvo, os dados e o número de unks.
source_tokenizer = Vectorizer(source_tokenizer_path, data=data[:, 0], n_unks=n_unks)
target_tokenizer = Vectorizer(target_tokenizer_path, data=data[:, 1], n_unks=n_unks)
print('source vocab: ', source_tokenizer.vocab_size)
print('target vocab: ', target_tokenizer.vocab_size)

# uso de vobaculário extra
if extra_vocab:
    punctuation = {'!', '?', '"', "\\", "'", "#", "%", "&", "(", ")", "*", "+", "-", ".", 
                   "/", ":", ";", "<", "=", ">", "@", "[", "]", "^", "_", "`", "~", "r$",
                   ","}
    print('Usando vocabulaŕio extra')
    new_data = restrict_sentences(parallel_data=data,
                                  source_seq_length=settings.NMT_TRAIN_CONFIG['model_structure']['max_source_length_sentence'],
                                  target_seq_length=1)
    
    # limpando tokens do source
    source_new_data = preprocess.clean_source(list(new_data[:, 0]))
    source_new_data = preprocess.clean_source(source_new_data) # necessário 2x para repetição de pontuação
    source_new_data_cleaned = []
    for sentence in source_new_data:
        clean_sentence = []
        for w in sentence.split():
            if w not in punctuation:
                w_clean = ''.join([c if c not in punctuation else ' ' for c in w])
                if w_clean.replace(' ', ''):
                    clean_sentence.append(w_clean)
        if clean_sentence:
            source_new_data_cleaned.append(' '.join(clean_sentence))
    
    # atualizando tokens
    source_tokenizer.update_vocab(source_new_data_cleaned)
    target_tokenizer.update_vocab(new_data[:, 1])
    print('source vocab depois: ', source_tokenizer.vocab_size)
    print('target vocab depois: ', target_tokenizer.vocab_size)

# Aqui nós utilizamos nossos tokenizers para transformar texto em sequências numéricas
source_sequences = source_tokenizer.text_to_sequences(cleaned_train[:, 0])
target_sequences = target_tokenizer.text_to_sequences(cleaned_train[:, 1])

# Os dados de tests são utilizados no treinamento para cálculo do bleu a cada época.
test_sequences = (source_tokenizer.text_to_sequences(cleaned_test[:, 0]),
                  target_tokenizer.text_to_sequences(cleaned_test[:, 1]))

test_data_path = os.path.join(settings.JOB_DIR, f'test_data.csv')
train_data_path = os.path.join(settings.JOB_DIR, f'train_data.csv')

pd.DataFrame(cleaned_test).to_csv(test_data_path, sep='\t', index=False, header=False)
pd.DataFrame(cleaned_train).to_csv(train_data_path, sep='\t', index=False, header=False)

print(f'{test_data_path}\n{train_data_path}\nGerados!')

# source vocab:  33181
# target vocab:  12304
# Usando vocabulaŕio extra
# source vocab depois:  38492
# target vocab depois:  12304

source vocab:  11620
target vocab:  16467
Usando vocabulaŕio extra
source vocab depois:  16006
target vocab depois:  16467
/content/nmt/JOB_DIR/en_asl/test_data.csv
/content/nmt/JOB_DIR/en_asl/train_data.csv
Gerados!


# Treinamento do Modelo

Já dividimos o corpus, depois limpamos os dados e, por último, transformamos texto em sequência numérica

E agora?

Agora nós precisamos passar todos os dados para o nosso modelo

Nós utilizamos a arquitetura word2word com attention e nós chamamos ele carinhosamente de HtNmt (:

Essa etapa é basicamente enviar as informações que já definimos

Em breve uma explicação detalhada (:

In [ ]:
# Criamos uma instancia do modelo HtNmt
model = HtNmt(source_vocab_size=source_tokenizer.vocab_size,
              target_vocab_size=target_tokenizer.vocab_size,
              n_units=settings.NMT_TRAIN_CONFIG['model_structure']['n_units'],
              n_dim=settings.NMT_TRAIN_CONFIG['model_structure']['n_dim'],
              checkpoints_dir=settings.JOB_DIR,
              attention=settings.MODEL_STRUCTURE['attention'],
              optimizer=settings.MODEL_STRUCTURE['optimizer'],
              learning_rate=settings.MODEL_STRUCTURE['learning_rate'],
              decay=settings.MODEL_STRUCTURE['decay'],
              seq_length=n_unks
              )

# Treinamento do modelok
# Aqui nós acendemos a fogueira e aguardamos o modelo ficar no ponto (:
model.train(source_sequences, target_sequences, validation_data=test_sequences, epochs=settings.MODEL_STRUCTURE['epochs'],
            batch_size=settings.MODEL_STRUCTURE['batch_size'])

history = cj.CsvFile.read(os.path.join(settings.JOB_DIR, 'history.csv'))

pyplot.subplot(211)
pyplot.title('Loss Function', pad=-40)
pyplot.plot([float(i) for i in history['train_loss']], label='train')
pyplot.plot([float(i) for i in history['test_loss']], label='test')
pyplot.legend()

# Plot bleu
pyplot.subplot(212)
pyplot.title('Bleu Score', pad=-40)
pyplot.plot([float(i) for i in history['train_bleu']], label='train')
pyplot.plot([float(i) for i in history['test_bleu']], label='test')
pyplot.legend()
pyplot.savefig(os.path.join(settings.JOB_DIR, 'train_plot.png'))

# Testando o modelo

Depois do modelo treinado (os arquivos do modelo estão no JOB_DIR), nós precisamos e queremos testá-lo

## Preditores

Nós desenvolvemos várias técnicas de predição, que visa melhorar a tradução em dois quesitos, qualidade e performance,
de maneira ***beeemmm*** resumida, nós criamos técnicas de quebra de sentenças e predições em paralelo/concorrente.

Os principais são `BaselineNgramPredict` e `SequentialPredict`

Neste exemplo vamos utilizar o `BaselineNgramPredict`

Todos os preditores precisam ter acesso ao `modelo`, pois o modelo é controlado por eles, também é necessário enviar a métrica `network_trust`, que a grosso modo é a média das softmaxs.


> Caso você não esteja utilizando o colab, você pode utilizar a interface web para testar o modelo
```python
from nmt.api.main import app
app.run(host='127.0.0.1', port=5000)
```

### Gerar dicionário

In [ ]:
if use_dict:
    from nmt.core.predict.predictors import DictionaryPredictors
    from nmt.core.models import Dictionary

    dictionary_instance = Dictionary(job_dir) if settings.NMT_TRAIN_CONFIG['prediction_config']['use_dict'] else None
    
    dict_predictor = Translator(DictionaryPredictors(model, ntr), source_tokenizer,
                                           target_tokenizer, dictionary_instance,
                                           preprocessor=preprocess)

    dict_predictor.translate(['oi'], preprocessing=True)

156350 --> 134435
134435 --> 134435


### Teste de Qualidade

In [ ]:
if quality_test:
    from nmt.core.models import Dictionary
    
    from nmt.core.predict.predictors import SequentialPredict
    
    postprocessor = PostProcessing(settings.ID_GLOSA_DIC)
    model.load_weights()
    
    # Carrega ou treina os dados do dicionário
    dictionary_instance = Dictionary(job_dir) if settings.NMT_TRAIN_CONFIG['prediction_config']['use_dict'] else None
    
    def get_translator_with_ntr(ntr: float, model):
        predicts_config = [model, ntr]
        # Vamos configurar nosso controlador que chamamos carinhosamente de `Translator`
        # Esse cara tem acesso a tudo, Modelo, Tokenizer, Preditor e Preprocessador
        return Translator(SequentialPredict(*predicts_config), source_tokenizer,
                                        target_tokenizer,
                                        dictionary_instance=dictionary_instance,
                                        preprocessor=preprocess)

    # Aqui nós instanciamos o nosso(s) preditor(es), passando o(s) `ntr`(s) e a instância treinada do `modelo`
    # E calculamos o bleu salvando um arquivo .csv para cada instância do translator
    ntrs = [0.60, 0.70, 0.75]
    for ntr_, translator in map(lambda x: (x, get_translator_with_ntr(x, model)), ntrs):
        nmt.core.evaluation.quality_model(translator, test, settings.JOB_DIR, name_prefix=f'ntr_{ntr_}')

### Teste de Performance

In [ ]:
if performance_test:
    # Aqui calculamos a performance do modelo utilizando os dados de test
    predicts = dict(map(lambda x: (f'BaseLineNgram_{x}', get_translator_with_ntr(x, model)), ntrs))
    nmt.core.evaluation.performance(predicts, Corpus.distinct_from_parallel(test)[0],
                os.path.join(settings.JOB_DIR, f'prediction_performance_{settings.TARGET_LANGUAGE}.png'),
                verbose=True
                )

# Salvar Modelo

In [ ]:
if save_model:
    dest_path = f'/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/PROCESSOS/NMT/ANO_{year}/{source_language.upper()}-{target_language.upper()}/TREINAMENTOS/{year}_{month}/{day}/{hour_min}/'
    if os.path.exists(dest_path):
        assert input(f"'{dest_path}' já existe, prosseguir? (Digite 'sim')?'") == 'sim'
    else: 
        os.makedirs(dest_path)
    comment_path = os.path.join(job_dir, 'comment_of_train.txt')
    !echo '{comment_of_train}' > '{comment_path}'
    !cp '{job_dir}/'* '{dest_path}'
    print(f"Treinamente salve em '{dest_path}'")

Treinamente salve em './TREINAMENTOS/ANO_2021/PT-BZS/2021_06/02/08_54/pt_bzs'


In [ ]:
!zip -r new_job.zip JOB_DIR/pt_bzs

updating: JOB_DIR/pt_bzs/ (stored 0%)
updating: JOB_DIR/pt_bzs/test_target_freq_words.json (deflated 56%)
updating: JOB_DIR/pt_bzs/history.csv (deflated 50%)
updating: JOB_DIR/pt_bzs/best_decoder_weights.h5 (deflated 8%)
updating: JOB_DIR/pt_bzs/test_source_freq_words.json (deflated 70%)
updating: JOB_DIR/pt_bzs/comment_of_train.txt (deflated 24%)
updating: JOB_DIR/pt_bzs/train_bzs.align (deflated 70%)
updating: JOB_DIR/pt_bzs/train_pt.align (deflated 57%)
updating: JOB_DIR/pt_bzs/test_pt.align (deflated 55%)
updating: JOB_DIR/pt_bzs/nmt_config.json (deflated 46%)
updating: JOB_DIR/pt_bzs/latest_decoder_weights.h5 (deflated 8%)
updating: JOB_DIR/pt_bzs/.ipynb_checkpoints/ (stored 0%)
updating: JOB_DIR/pt_bzs/.ipynb_checkpoints/history-checkpoint.csv (deflated 50%)
updating: JOB_DIR/pt_bzs/.ipynb_checkpoints/pt-checkpoint.tk (deflated 54%)
updating: JOB_DIR/pt_bzs/.ipynb_checkpoints/nmt_config-checkpoint.json (deflated 46%)
updating: JOB_DIR/pt_bzs/.ipynb_checkpoints/bzs-checkpoint.tk (

# Realizar traduções

In [ ]:
custom_job_dir = './TREINAMENTOS/ANO_2021/PT-BZS/2021_03/30/19_41/pt_bzs'
src, trg = os.path.basename(custom_job_dir).split('_')
'-'.join([src, trg])

In [ ]:
NMT_CONFIG_CUSTOM = {
  "_comment": "Configuração geral do sistema - NEW",
  "SOURCE_LANGUAGE": src,
  "TARGET_LANGUAGE": trg,
  "ENVIRONMENT": "development",
  "CHOICE_PREDICTS": {
    "BaselineNgramPredict": "nmt.core.predict.predictors.BaselineNgramPredict"
  },
  "DEFAULT_PREDICTION": "BaselineNgramPredict",
  "MODEL_ARCHITECTURE": f"nmt.core.models.base.HtNmt",
  "EPOCHS": 30,
  "JOB_DIR": custom_job_dir,
  "CORPUS_DIR": "/content/CORPUS_DIR",
  "NORMAL_TRAIN": False,
  "PERFORMANCE_TEST": False,
  "USE_DICT": True,
  "ATTENTION": "ht_attention",
  "OPTIMIZER": 'adam',
  "LEARNING_RATE": 0,
  "DECAY": 0.0002,
  "N_DIM": 100,
  "N_UNITS": 128,
  "BATCH_SIZE": 32,
  "EXTRA_VOCAB": False,
  "NTR": 0.75,
  "LOG_ARGMAX_LENGTH": 3,
  "MAX_SOURCE_LENGTH_SENTENCE": 10,
  "MAX_TARGET_LENGTH_SENTENCE": 10,
  "PARSER": 10,
}

with open('./nmt/nmt/config/config.json', 'w', encoding='utf-8') as f:
    json.dump(NMT_CONFIG_CUSTOM, f, indent=True)

import nmt

In [ ]:
# carregar modelo
from nmt import settings
from nmt.core.models.base import HtNmt
from nmt.core.tokenizer.tokenizer import Vectorizer
from nmt.core.predict.predictors import BaselineNgramPredict
from nmt.core.predict.controller import Translator
from nmt.core.pln import PreProcessing, PostProcessing

n_unks = min(settings.MODEL_STRUCTURE["max_source_length_sentence"],
             settings.MODEL_STRUCTURE["max_target_length_sentence"])


source_tokenizer_path = os.path.join(settings.JOB_DIR, '{}.tk'.format(settings.SOURCE_LANGUAGE))
target_tokenizer_path = os.path.join(settings.JOB_DIR, '{}.tk'.format(settings.TARGET_LANGUAGE))
    
source_tokenizer = Vectorizer(source_tokenizer_path, n_unks=n_unks)
target_tokenizer = Vectorizer(target_tokenizer_path, n_unks=n_unks)

model = HtNmt(source_vocab_size=source_tokenizer.vocab_size,
              target_vocab_size=target_tokenizer.vocab_size,
              n_units=settings.NMT_TRAIN_CONFIG['model_structure']['n_units'],
              n_dim=settings.NMT_TRAIN_CONFIG['model_structure']['n_dim'],
              checkpoints_dir=settings.JOB_DIR,
              attention=settings.MODEL_STRUCTURE['attention'],
              optimizer=settings.MODEL_STRUCTURE['optimizer'],
              learning_rate=settings.MODEL_STRUCTURE['learning_rate'],
              decay=settings.MODEL_STRUCTURE['decay'],
              seq_length=n_unks)

preprocessor = PreProcessing.build(settings.PLN_CONFIG)
postprocessor = PostProcessing(settings.ID_GLOSA_DIC)

predicts_config = [model, settings.PREDICTION_CONFIG['network_trust']]
translator = Translator(BaselineNgramPredict(*predicts_config), source_tokenizer,
                        target_tokenizer,
                        preprocessor=preprocessor,
                        postprocessor=postprocessor)

model.load_weights()

In [ ]:
# realizar traduções
sentence = ''
while sentence.lower() != 'exit':
    sentence = input("Traduzir ('exit' p/sair): ")
    translation = ' '.join(translator.translate([sentence],  preprocessing=True, postprocessing=True))
    print(translation)